# Time-series Data Forecasting Methods

Demonstrate forecasting methods on real data:

* Exponencial smoothing
* Holt method
* Holt Winters method

In [1]:
"""
Core library imports
"""
import json

In [2]:
"""
pandas: pandas is a library providing high-performance,
easy-to-use data structures and data analysis tools for
the Python programming language.
http://pandas.pydata.org/
"""
import pandas as pd

In [3]:
"""
libgraphite: Python library for querying metrics from
Graphite into Pandas DataFrames.
https://github.com/direvius/libgraphite
"""
import libgraphite as lg

In [4]:
"""
Bokeh: Bokeh is a Python interactive visualization library
that targets modern web browsers for presentation.
http://bokeh.pydata.org/
"""
from bokeh.plotting import figure, show
from bokeh.models import BoxAnnotation, ColumnDataSource
from bokeh.io import output_notebook, vform

In [5]:
"""
pycast: pycast aims to provide a python module supporting basic
and advanced smoothing and forecasting methods that can be used
on time series data.
http://pythonhosted.org/pycast/
"""
from pycast.common.timeseries import TimeSeries
from pycast.common.json_encoder import PycastEncoder
from pycast.methods.exponentialsmoothing import ExponentialSmoothing, \
     HoltMethod, HoltWintersMethod
from pycast.errors import SymmetricMeanAbsolutePercentageError as SMAPE

In [6]:
#Analysis Parameters
valuesToForecast = 6
smoothingFactor = float(0.2)
trendSmoothingFactor = float(0.3)
seasonSmoothingFactor = float(0.4)
seasonLength = 24
#Visualization Parameters
figureTools = "" # "pan,wheel_zoom,box_zoom,reset,save"

In [7]:
#Data for Analysis
server = 'http://graphite-server.iot.tcpcloud.eu:80'
target = 'alias(transformNull(iot.doppler.pi03.dvb.passage), "pi03-dvb")'
dt_data = lg.Query(server) \
            .target(target) \
            .pfrom('20151204') \
            .puntil('20151213') \
            .execute()
dt_data_dict = dt_data.to_dict()
dt_data_list = []
for dt_stamp, dt_value in dt_data_dict['pi03-dvb'].iteritems():
    dt_data_list.append([dt_stamp, dt_value])
original = TimeSeries.from_twodim_list(dt_data_list)
original.normalize("hour", fusionMethod = "sum")

AttributeError: 'NoneType' object has no attribute 'to_dict'

In [ ]:
#Data Visualization Initialization
output_notebook()

In [ ]:
pd_original = pd.read_json(json.dumps(original, cls=PycastEncoder))
print pd_original

In [ ]:
# Plot Raw Data
p = figure(x_axis_type="datetime", tools=figureTools)
p.line(pd_original[0], pd_original[1],
       line_color="black", line_width=2, legend="Original Time-Series")

p.title = "Original Time-series"
p.xgrid[0].grid_line_color=None
p.ygrid[0].grid_line_alpha=0.5
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Traffic'
p.plot_width = 900
p.plot_height = 350
show(p)

In [ ]:
#Perform Exponential Smoothing
es = ExponentialSmoothing(smoothingFactor = smoothingFactor,
                          valuesToForecast = valuesToForecast)
smoothed = es.execute(original)
es_smoothed = pd.read_json(json.dumps(smoothed, cls=PycastEncoder))
print es_smoothed

In [ ]:
# Plot Exponential Smoothing
p = figure(x_axis_type="datetime", tools=figureTools)
p.line(pd_original[0], pd_original[1],
       line_color="black", line_width=2, legend="Original Time-Series")
p.line(es_smoothed[0], es_smoothed[1],
       line_color="blue", line_width=1, legend="Exponencial Smoothing")

p.title = "Exponential Smoothing"
p.xgrid[0].grid_line_color=None
p.ygrid[0].grid_line_alpha=0.5
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Traffic'
p.plot_width = 900
p.plot_height = 350
p.legend.orientation = "top_left"
show(p)

In [ ]:
#Perform Holt Method
hm = HoltMethod(smoothingFactor = smoothingFactor,
                trendSmoothingFactor = trendSmoothingFactor,
                valuesToForecast = valuesToForecast)
smoothed = hm.execute(original)
hm_smoothed = pd.read_json(json.dumps(smoothed, cls=PycastEncoder))
print hm_smoothed

In [ ]:
# Plot Holt Method
p = figure(x_axis_type="datetime", tools=figureTools)
p.line(pd_original[0], pd_original[1],
       line_color="black", line_width=2, legend="Original Time-Series")
p.line(hm_smoothed[0], hm_smoothed[1],
       line_color="red", line_width=1, legend="Holt Method")

p.title = "Holt Method"
p.xgrid[0].grid_line_color=None
p.ygrid[0].grid_line_alpha=0.5
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Traffic'
p.plot_width = 900
p.plot_height = 350
p.legend.orientation = "top_left"
show(p)

In [ ]:
#Perform Holt-Winters Method
hwm = HoltWintersMethod(smoothingFactor = smoothingFactor,
                        trendSmoothingFactor = trendSmoothingFactor,
                        seasonSmoothingFactor = seasonSmoothingFactor,
                        seasonLength = seasonLength,
                        valuesToForecast = valuesToForecast)
smoothed = hwm.execute(original)
hwm_smoothed = pd.read_json(json.dumps(smoothed, cls=PycastEncoder))
print hwm_smoothed

In [ ]:
# Plot all methods
p = figure(x_axis_type="datetime", tools=figureTools)
p.line(pd_original[0], pd_original[1],
       line_color="black", line_width=2, legend="Original Time-Series")
p.line(hwm_smoothed[0], hwm_smoothed[1],
       line_color="green", line_width=1, legend="Holt-Winters Method")

p.title = "Holt-Winters Method"
p.xgrid[0].grid_line_color=None
p.ygrid[0].grid_line_alpha=0.5
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Traffic'
p.plot_width = 900
p.plot_height = 350
p.legend.orientation = "top_left"
show(p)

In [ ]:
# Print error
error = SMAPE()
error.initialize(original, smoothed)
error = round(error.get_error(), 3)
print error

###data = json.dumps(original, cls=PycastEncoder)
###original = TimeSeries.from_twodim_list(data)

In [ ]:
# Plot all methods
p = figure(x_axis_type="datetime", tools=figureTools)
p.line(pd_original[0], pd_original[1],
       line_color="black", line_width=2, legend="Original Time-Series")
p.line(es_smoothed[0], es_smoothed[1],
       line_color="blue", line_width=1, legend="Exponencial Smoothing")
p.line(hm_smoothed[0], hm_smoothed[1],
       line_color="red", line_width=1, legend="Holt Method")
p.line(hwm_smoothed[0], hwm_smoothed[1],
       line_color="green", line_width=1, legend="Holt-Winters Method")

p.title = "Time-Series Predictive Analysis Comparison"
p.xgrid[0].grid_line_color=None
p.ygrid[0].grid_line_alpha=0.5
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Traffic'
p.plot_width = 900
p.plot_height = 350
p.legend.orientation = "top_left"
show(p)